<a href="https://colab.research.google.com/github/juanpi19/medium-articles/blob/main/other/statsig_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [288]:
import pandas as pd
import numpy as np
from scipy.stats import norm


In [3]:
assignments_df = pd.read_csv("assignments_sample_data.csv")
metrics_df = pd.read_csv("metrics_sample_data.csv")

In [323]:
assignments_df.head()

,user_id,time,group,country,platform
0,56bb3bb1-b180-419c-8418-dbf9634f1e85,2025-01-31 21:47:31.489146 UTC,GPT-4,US,Desktop App
1,ac0e2cbc-b0a3-42c0-a808-78096dd50ddc,2025-01-31 21:47:31.489146 UTC,GPT-4o,CA,Desktop Web
2,fb516003-fae5-43f0-9b13-8e84b4fb272b,2025-01-31 21:47:31.489146 UTC,GPT-4o,MX,Mobile App
3,077a1f73-b743-43a6-aa0c-7c30b8899071,2025-01-31 21:47:31.489146 UTC,GPT-3.5,MX,Mobile Web
4,49544e21-44a0-4995-9102-8fa27fb5d0dc,2025-01-31 21:47:31.489146 UTC,GPT-3.5,CA,Desktop App


In [324]:
metrics_df.head()

,user_id,time,metric_name,metric_value
0,64b97ff3-cbb0-42a6-adc6-fa63ccb964b0,2025-01-31,Messages Sent,1.0
1,cd5a0eac-66bc-4050-9d90-3765f152ff6b,2025-01-31,Messages Sent,1.0
2,f68421aa-f7d0-4c55-bf7b-da54d5b2e424,2025-01-31,Messages Sent,1.0
3,0f7b107f-04c2-4c75-8288-e568e5203a23,2025-01-31,Messages Sent,1.0
4,b7e03080-6919-406b-9eb3-15280b62e422,2025-01-31,Messages Sent,1.0


In [36]:
merged_df = pd.merge(metrics_df, assignments_df, how='inner', on='user_id')
merged_df.head()

,user_id,time_x,metric_name,metric_value,time_y,group,country,platform
0,64b97ff3-cbb0-42a6-adc6-fa63ccb964b0,2025-01-31,Messages Sent,1.0,2025-01-31 21:47:31.489146 UTC,GPT-3.5,MX,Desktop App
1,cd5a0eac-66bc-4050-9d90-3765f152ff6b,2025-01-31,Messages Sent,1.0,2025-01-31 21:47:31.489146 UTC,GPT-4o,CA,Desktop App
2,f68421aa-f7d0-4c55-bf7b-da54d5b2e424,2025-01-31,Messages Sent,1.0,2025-01-31 21:47:31.489146 UTC,GPT-3.5,CA,Mobile Web
3,0f7b107f-04c2-4c75-8288-e568e5203a23,2025-01-31,Messages Sent,1.0,2025-01-31 21:47:31.489146 UTC,GPT-3.5,CA,Mobile App
4,b7e03080-6919-406b-9eb3-15280b62e422,2025-01-31,Messages Sent,1.0,2025-01-31 21:47:31.489146 UTC,GPT-3.5,CA,Mobile Web


In [44]:
assignments_df['group'].value_counts()

,count
group,
GPT-3.5,3622
GPT-4o,3605
GPT-4,3454


# Reworking the Math

In [187]:
merged_df.head()

,user_id,time_x,metric_name,metric_value,time_y,group,country,platform
0,64b97ff3-cbb0-42a6-adc6-fa63ccb964b0,2025-01-31,Messages Sent,1.0,2025-01-31 21:47:31.489146 UTC,GPT-3.5,MX,Desktop App
1,cd5a0eac-66bc-4050-9d90-3765f152ff6b,2025-01-31,Messages Sent,1.0,2025-01-31 21:47:31.489146 UTC,GPT-4o,CA,Desktop App
2,f68421aa-f7d0-4c55-bf7b-da54d5b2e424,2025-01-31,Messages Sent,1.0,2025-01-31 21:47:31.489146 UTC,GPT-3.5,CA,Mobile Web
3,0f7b107f-04c2-4c75-8288-e568e5203a23,2025-01-31,Messages Sent,1.0,2025-01-31 21:47:31.489146 UTC,GPT-3.5,CA,Mobile App
4,b7e03080-6919-406b-9eb3-15280b62e422,2025-01-31,Messages Sent,1.0,2025-01-31 21:47:31.489146 UTC,GPT-3.5,CA,Mobile Web


# Mean

In [189]:
agg_df = merged_df.groupby(['group', 'metric_name']).agg(
    metric_value_sum = ('metric_value','sum'),
    ).reset_index().sort_values(by='metric_name')
agg_df

,group,metric_name,metric_value_sum
0,GPT-3.5,Messages Sent,2622.0
5,GPT-4,Messages Sent,2092.0
10,GPT-4o,Messages Sent,2568.0
1,GPT-3.5,Page Loads,27561.0
6,GPT-4,Page Loads,24388.0
11,GPT-4o,Page Loads,28227.0
2,GPT-3.5,Purchase Revenue,92311.0
7,GPT-4,Purchase Revenue,123766.0
12,GPT-4o,Purchase Revenue,131483.0
3,GPT-3.5,Visit,4894.0


In [191]:
num_assingments_df = assignments_df['group'].value_counts().reset_index()
final_agg_df = pd.merge(agg_df, num_assingments_df, how='left', on='group')
final_agg_df['metric_mean'] = final_agg_df['metric_value_sum']/final_agg_df['count']
final_agg_df

,group,metric_name,metric_value_sum,count,metric_mean
0,GPT-3.5,Messages Sent,2622.0,3622,0.723909
1,GPT-4,Messages Sent,2092.0,3454,0.605675
2,GPT-4o,Messages Sent,2568.0,3605,0.712344
3,GPT-3.5,Page Loads,27561.0,3622,7.609332
4,GPT-4,Page Loads,24388.0,3454,7.060799
5,GPT-4o,Page Loads,28227.0,3605,7.829958
6,GPT-3.5,Purchase Revenue,92311.0,3622,25.486195
7,GPT-4,Purchase Revenue,123766.0,3454,35.832658
8,GPT-4o,Purchase Revenue,131483.0,3605,36.472399
9,GPT-3.5,Visit,4894.0,3622,1.351187


# Standard Error

In [204]:
var_df = pd.merge(merged_df, final_agg_df[['group', 'metric_name', 'metric_mean']], how='left', on=['group', 'metric_name'])
var_df['var_numerator'] = (var_df['metric_value'] - var_df['metric_mean'])**2


var_agg_df = var_df.groupby(['group','metric_name']).agg({'var_numerator': 'sum', 'metric_mean': 'first'}).reset_index()
var_agg_df = pd.merge(var_agg_df, final_agg_df[['group', 'metric_name', 'count']], how='left', on=['group', 'metric_name'])
var_agg_df['sample_std'] = np.sqrt(var_agg_df['var_numerator'] / (var_agg_df['count'] - 1))
var_agg_df['standard_error'] = var_agg_df['sample_std'] / np.sqrt(var_agg_df['count'])
var_agg_df.sort_values(by='metric_name')


,group,metric_name,var_numerator,metric_mean,count,sample_std,standard_error
0,GPT-3.5,Messages Sent,1.998646e+02,0.723909,3622,0.234938,0.003904
5,GPT-4,Messages Sent,3.252904e+02,0.605675,3454,0.306929,0.005222
10,GPT-4o,Messages Sent,2.124917e+02,0.712344,3605,0.242817,0.004044
1,GPT-3.5,Page Loads,8.972683e+04,7.609332,3622,4.977909,0.082713
6,GPT-4,Page Loads,6.455125e+04,7.060799,3454,4.323685,0.073569
11,GPT-4o,Page Loads,9.221361e+04,7.829958,3605,5.058306,0.084247
2,GPT-3.5,Purchase Revenue,4.010632e+06,25.486195,3622,33.280677,0.552991
7,GPT-4,Purchase Revenue,4.107389e+06,35.832658,3454,34.489319,0.586845
12,GPT-4o,Purchase Revenue,4.328224e+06,36.472399,3605,34.654727,0.577178
3,GPT-3.5,Visit,6.035890e+02,1.351187,3622,0.408278,0.006784


# P-values | Confidence Intervals

In [321]:
def calculate_p_values(a, b, metric, df):

  a_mean, b_mean = df[(df['group'] == a) & (df['metric_name'] == metric)]['metric_mean'], df[(df['group'] == b) & (df['metric_name'] == metric)]['metric_mean']
  a_ste, b_ste = df[(df['group'] == a) & (df['metric_name'] == metric)]['standard_error'], df[(df['group'] == b) & (df['metric_name'] == metric)]['standard_error']

  pool_ste = np.sqrt(a_ste.iloc[0]**2 + b_ste.iloc[0]**2 / 2)

  z = (b_mean.iloc[0] - a_mean.iloc[0]) / pool_ste

  p_value_one_tailed = 1 - norm.cdf(z)

  return p_value_one_tailed, z



# Confidence interval

def calculate_confidence_interval(a, b, metric, df):

  a_mean, b_mean = df[(df['group'] == a) & (df['metric_name'] == metric)]['metric_mean'], df[(df['group'] == b) & (df['metric_name'] == metric)]['metric_mean']
  a_ste, b_ste = df[(df['group'] == a) & (df['metric_name'] == metric)]['standard_error'], df[(df['group'] == b) & (df['metric_name'] == metric)]['standard_error']

  pool_ste = np.sqrt(a_ste.iloc[0]**2 + b_ste.iloc[0]**2 / 2)
  margin_of_error = pool_ste * 1.96

  lower_ci = (b_mean.iloc[0] - a_mean.iloc[0]) - margin_of_error
  upper_ci = (b_mean.iloc[0] - a_mean.iloc[0]) + margin_of_error

  return round(lower_ci,2), round(upper_ci,2)



In [275]:
final_df = var_agg_df.copy()
final_df.head()

,group,metric_name,var_numerator,metric_mean,count,sample_std,standard_error
0,GPT-3.5,Messages Sent,1.998646e+02,0.723909,3622,0.234938,0.003904
1,GPT-3.5,Page Loads,8.972683e+04,7.609332,3622,4.977909,0.082713
2,GPT-3.5,Purchase Revenue,4.010632e+06,25.486195,3622,33.280677,0.552991
3,GPT-3.5,Visit,6.035890e+02,1.351187,3622,0.408278,0.006784
4,GPT-3.5,Week 1 Purchase,3.845630e+06,23.066538,3622,32.588882,0.541496


In [322]:
variants = ['GPT-4o', 'GPT-4']
METRIC = 'Purchase Revenue'
control = 'GPT-3.5'

for variant in variants:
  p_value, z = calculate_p_values(a=control, b=variant, metric=METRIC, df=final_df)
  ci = calculate_confidence_interval(a=control, b=variant, metric=METRIC, df=final_df)
  print(f"{control} vs. {variant}:")
  print(f"P-value: {p_value} | z-score: {round(z,2)} ")
  print(f"Confidence Interval: {ci}")
  print("*"*50)




GPT-3.5 vs. GPT-4o:
P-value: 0.0 | z-score: 15.98 
Confidence Interval: (np.float64(9.64), np.float64(12.33))
**************************************************
GPT-3.5 vs. GPT-4:
P-value: 0.0 | z-score: 14.97 
Confidence Interval: (np.float64(8.99), np.float64(11.7))
**************************************************
